### JsonOutputParser
* 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 parser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model="gpt-4o")

In [3]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")
    
output_parser = JsonOutputParser(pydantic_object=Topic)

In [4]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [7]:
from langchain_core.prompts import ChatPromptTemplate

question = "점심먹고 졸린 이유를 알려주세요."

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 대답하세요."),
    ("user", "Format:\n{format_instructions}\nQuestion:\n{question}")
])

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

In [8]:
chain = prompt | model | output_parser

answer = chain.invoke({"question": question})

type(answer)
print(answer)

{'description': '점심 식사 후 졸림은 주로 소화 과정에서 혈액이 소화기관으로 집중되면서 뇌로 가는 혈류가 줄어들기 때문입니다. 또한, 식사 후 혈당 수치가 급격히 변동하면서 졸음을 유발할 수 있습니다.', 'hashtags': '#점심졸림 #소화과정'}


### Pydantic을 사용하지않고 JsonOutputParser 사용

In [13]:
output_parser = JsonOutputParser()

question = "졸음이 쏟아질 때 졸음을 이길 수 있는 방법을 알려주세요. 방법에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "질문에 친절하고 상냥하고 간결하게 대답하세요."),
    ("user", "Format:\n{format_instructions}\nQuestion:\n{question}")
])

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | model | output_parser

response = chain.invoke({"question": question})

print(response)

{'methods': [{'description': '잠시 일어나서 스트레칭을 하거나 가벼운 운동을 하여 몸을 깨운다.', 'hashtags': ['#스트레칭', '#운동', '#활력']}, {'description': '차가운 물로 얼굴을 씻어 졸음을 깨운다.', 'hashtags': ['#차가운물', '#세수', '#리프레시']}, {'description': '커피나 녹차 같은 카페인이 들어간 음료를 마신다.', 'hashtags': ['#커피', '#녹차', '#카페인']}, {'description': '잠시 창문을 열어 신선한 공기를 마시며 환기한다.', 'hashtags': ['#환기', '#신선한공기', '#리프레시']}, {'description': '짧은 낮잠을 자서 피로를 회복한다.', 'hashtags': ['#낮잠', '#피로회복', '#리프레시']}, {'description': '간단한 간식을 먹어 혈당을 올려 에너지를 보충한다.', 'hashtags': ['#간식', '#혈당', '#에너지']}]}
